In [1]:
import tensorflow as tf
import numpy as np

import itertools
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from src.models import lstm, tcn
from src.utils import auxiliary_plots, metrics
from src.utils.print_functions import notify_slack
from src.preprocessing import normalization, data_generation
import time

2023-12-15 12:23:14.690900: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 12:23:14.690933: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-15 12:23:14.690959: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-15 12:23:14.697741: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from src.experiments import run_experiments
import os

WEBHOOK = os.environ.get('webhook_slack')
print(WEBHOOK)

METRICS = ['mse', 'rmse', 'nrmse', 'mae', 'wape', 'mpe', 'mape', 'mdape', 'smape', 'smdape',
           'mase', 'rmspe', 'rmsse', 'mre', 'rae', 'mrae', 'std_ae', 'std_ape']

TCN_PARAMS = {
    'nb_filters': [32, 64, 128],
    'kernel_size': [2, 3, 4, 5, 6],
    'nb_stacks': [1, 2, 3, 4, 5],
    'dilations': [[1, 2, 4], [1, 2, 4, 8], [1, 2, 4, 8, 16], [1, 2, 4, 8, 16, 32], [1, 2, 4, 8, 16, 32, 64],
                  [1, 3, 9], [1, 3, 9, 27], [1, 3, 9, 27, 81],
                  [1, 3, 6], [1, 3, 6, 12], [1, 3, 6, 12, 24], [1, 3, 6, 12, 24], [1, 3, 6, 12, 24, 48],
                  [1, 5, 7], [1, 5, 7, 14], [1, 5, 7, 14, 28], [1, 5, 7, 14, 28, 56]],
    'dropout_rate': [0],
}

TCN_PARAMS = {
    # 'nb_filters': [32, 64, 128],
    # 'kernel_size': [2, 3, 4, 5],
    # 'nb_stacks': [1, 2, 3, 4],
    # 'dilations': [[1, 2, 4], [1, 2, 4, 8], [1, 2, 4, 8, 16], [1, 2, 4, 8, 16, 32], [1, 2, 4, 8, 16, 32, 64],
    #               [1, 3, 6], [1, 3, 6, 12], [1, 5, 7], [1, 5, 7, 14]],
    # 'dropout_rate': [0],
    'nb_filters': [128],
    'kernel_size': [6],
    'nb_stacks': [1, 2, 3, 4],
    'dilations': [[1, 3, 6, 12]],
    'dropout_rate': [0],
}
LSTM_PARAMS = {
    'num_stack_layers': [1, 2, 3],
    'units': [32, 64, 128],
    'dropout': [0]
}

FORECAST_HORIZON = 24
PAST_HISTORY = [144]

BATCH_SIZE = [128]
BUFFER_SIZE = 10000

EPOCHS = [50]

_GPU_NUMBER = None

# Electric demand forecasting
#run_experiments(train_file_name='data/hourly_20140102_20191101_train.csv',
#                test_file_name='data/hourly_20140102_20191101_test.csv',
#                result_file_name='experimental_results_electricity.csv',
#                forecast_horizon=FORECAST_HORIZON,
#                past_history_ls=PAST_HISTORY,
#                batch_size_ls=BATCH_SIZE,
#                epochs_ls=EPOCHS,
#                tcn_params=TCN_PARAMS,
#                lstm_params=None,
#                gpu_number=_GPU_NUMBER,
#                metrics_ls=METRICS,
#                buffer_size=1000,
#                seed=1,
#                show_plots=False,
#                webhook=WEBHOOK,
#                validation_size=0.)

# Electric vehicle power consumption forecasting
# run_experiments(train_file_name='data/CECOVEL_train.csv',
#                 test_file_name='data/CECOVEL_test.csv',
#                 result_file_name='files/results/experimental_results_EV.csv',
#                 forecast_horizon=FORECAST_HORIZON,
#                 past_history_ls=PAST_HISTORY,
#                 batch_size_ls=BATCH_SIZE,
#                 epochs_ls=EPOCHS,
#                 tcn_params=TCN_PARAMS,
#                 lstm_params=LSTM_PARAMS,
#                 gpu_number=None,
#                 metrics_ls=METRICS,
#                 buffer_size=1000,
#                 seed=1,
#                 show_plots=False,
#                 webhook=WEBHOOK,
#                 validation_size=0.)


None


In [3]:
seed = 1
result_file_name= 'experimental_results_electricity.csv'
metrics_ls=METRICS
train_file_name='data/hourly_20140102_20191101_train.csv'
test_file_name='data/hourly_20140102_20191101_test.csv'
result_file_name='experimental_results_electricity.csv'
forecast_horizon=FORECAST_HORIZON
past_history_ls=PAST_HISTORY
batch_size_ls=BATCH_SIZE
epochs_ls=EPOCHS
tcn_params=TCN_PARAMS
lstm_params=None
gpu_number=_GPU_NUMBER
metrics_ls=METRICS
buffer_size=1000
seed=1
show_plots=False
webhook=WEBHOOK
validation_size=0.1

In [4]:
#(train_file_name, test_file_name, result_file_name, forecast_horizon, past_history_ls, batch_size_ls,
#epochs_ls, tcn_params=TCN_PARAMS, lstm_params=LSTM_PARAMS, gpu_number=None, metrics_ls=METRICS,
#buffer_size=1000, seed=1, show_plots=False, webhook=None, validation_size=0.2)


tf.random.set_seed(seed)
np.random.seed(seed)


In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
device_name = str(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
# Write result csv header
current_index = 0
try:
    with open(result_file_name, 'r') as resfile:
        current_index = sum(1 for line in resfile) - 1
except IOError:
    pass
print('CURRENT INDEX', current_index)
if current_index == 0:
    with open(result_file_name, 'w') as resfile:
        resfile.write(';'.join([str(a) for a in
                                ['MODEL', 'MODEL_DESCRIPTION', 'FORECAST_HORIZON', 'PAST_HISTORY', 'BATCH_SIZE',
                                 'EPOCHS'] +
                                metrics_ls + ['val_' + m for m in metrics_ls] + ['loss', 'val_loss',
                                                                                 'Execution_time',
                                                                                 'Device']]) + "\n")

CURRENT INDEX -1


In [7]:
# Read train file
with open(train_file_name, 'r') as datafile:
    ts_train = datafile.readlines()[1:]  # skip the header
    ts_train = np.asarray([np.asarray(l.rstrip().split(',')[0], dtype=np.float32) for l in ts_train])
    ts_train = np.reshape(ts_train, (ts_train.shape[0],))

# Read test data file
with open(test_file_name, 'r') as datafile:
    ts_test = datafile.readlines()[1:]  # skip the header
    ts_test = np.asarray([np.asarray(l.rstrip().split(',')[0], dtype=np.float32) for l in ts_test])
    ts_test = np.reshape(ts_test, (ts_test.shape[0],))

In [8]:
# Train/validation split
TRAIN_SPLIT = int(ts_train.shape[0] * (1 - validation_size))
print(ts_train.shape, TRAIN_SPLIT)
# Normalize training data
norm_params = normalization.get_normalization_params(ts_train[:TRAIN_SPLIT])
ts_train = normalization.normalize(ts_train, norm_params)
# Normalize test data with train params
ts_test = normalization.normalize(ts_test, norm_params)

(245376,) 220838


In [9]:
i = 0
index_1, total_1 = 0, len(list(itertools.product(past_history_ls, batch_size_ls, epochs_ls)))


In [10]:
list(itertools.product(past_history_ls, batch_size_ls, epochs_ls))

[(144, 128, 50)]

In [18]:
for past_history, batch_size, epochs in tqdm(list(itertools.product(past_history_ls, batch_size_ls, epochs_ls))):
    index_1 += 1
    # Get x and y for training and validation
    #x_train, y_train = data_generation.univariate_data(ts_train, 0, TRAIN_SPLIT, past_history, forecast_horizon)
    #x_val, y_val = data_generation.univariate_data(ts_train, TRAIN_SPLIT - past_history, ts_train.shape[0],
    #                                               past_history, forecast_horizon)
    #print(x_train.shape, y_train.shape, '\n', x_val.shape, y_val.shape)
    # Get x and y for test data
    #x_test, y_test = data_generation.univariate_data(ts_test, 0, ts_test.shape[0], past_history, forecast_horizon)

    # Convert numpy data to tensorflow dataset
    #train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).cache().shuffle(buffer_size).batch(
    #    batch_size).repeat()
    #val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(
    #    batch_size).repeat() if validation_size > 0 else None
    #test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
    
    save_best = tf.keras.callbacks.ModelCheckpoint(filepath=f'best-model-bs-{batch_size}-ph-{past_history}.ckpt',save_best_only=True, monitor='val_loss')
    
    # Create models
    model_list = {}
    model_description_list = {}
    if tcn_params is not None:
        model_list = {'TCN_{}'.format(j): (tcn, [x_train.shape, forecast_horizon, 'adam', 'mae', *params]) for
                      j, params in
                      enumerate(itertools.product(*tcn_params.values())) if
                      params[1] * params[2] * params[3][-1] == past_history}
        model_description_list = {'TCN_{}'.format(j): str(dict(zip(tcn_params.keys(), params))) for j, params in
                                  enumerate(itertools.product(*tcn_params.values())) if
                                  params[1] * params[2] * params[3][-1] == past_history}
    if lstm_params is not None:
        model_list = {**model_list,
                      **{'LSTM_{}'.format(j): (lstm, [x_train.shape, forecast_horizon, 'adam', 'mae', *params]) for
                         j, params in
                         enumerate(itertools.product(*lstm_params.values()))}}
        model_description_list = {**model_description_list,
                                  **{'LSTM_{}'.format(j): str(dict(zip(lstm_params.keys(), params))) for j, params
                                     in
                                     enumerate(itertools.product(*lstm_params.values()))}}

    steps_per_epoch = int(np.ceil(x_train.shape[0] / batch_size))
    validation_steps = None

    index_2, total_2 = 0, len(model_list.keys())
    for model_name, (model_function, params) in tqdm(model_list.items(), position=1):
        index_2 += 1
        i += 1
        if i <= current_index:
            continue
        start = time.time()
        model = model_function(*params)
        print(model.summary())


  0%|                                                                             | 0/1 [00:00<?, ?it/s]

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 144, 1)]          0         
                                                                 
 tcn (TCN)                   (None, 128)               1477632   
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
Total params: 1481328 (5.65 MB)
Trainable params: 1481328 (5.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________



100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.65it/s]

None


In [19]:
model_function

<function src.models.tcn(input_shape, output_size=1, optimizer='adam', loss='mae', nb_filters=64, kernel_size=2, nb_stacks=1, dilations=[1, 2, 4, 8, 16, 32], dropout_rate=0, use_skip_connections=True, use_batch_norm=False, activation='linear', return_sequences=False, dense_layers=[], dense_dropout=0.0)>

In [20]:
params

[(220671, 144, 1), 24, 'adam', 'mae', 128, 6, 2, [1, 3, 6, 12], 0]

In [21]:
tcn

<function src.models.tcn(input_shape, output_size=1, optimizer='adam', loss='mae', nb_filters=64, kernel_size=2, nb_stacks=1, dilations=[1, 2, 4, 8, 16, 32], dropout_rate=0, use_skip_connections=True, use_batch_norm=False, activation='linear', return_sequences=False, dense_layers=[], dense_dropout=0.0)>